# Simple Linear versus Ridge Regression 

## Summary

Linear regression was one of the earliest types of regression that studied rigorously. It is widely used for predicting continuous values. For that reason, we used the “Boston house prices dataset” to address this regression problem. The model takes into account 13 attributes to try to predict the price of a house in Boston.

The program itself should be run linearly as to ensure the appropriate libraries are loaded and all data pre-processing occurs before we run any regressions. We can then run the regressions and see the performance of different regression parameters in the 11th and 12th code block. By observing the train and test error, we can determine that the best performing model is a Linear Regression with Polynomial Features of Second Order. Since it contains 105 parameters, it would be impractical for me to list them in this report, but they can be found in the 13th code block of the program. We can see that the parameters vary across many degrees of magnitude all the way from e-05 to e+01.

## Step 1:  Getting, understanding, and preprocessing the dataset

We first import the standard libaries and some libraries that will help us scale the data and perform some "feature engineering" by transforming the data into $\Phi_2({\bf x})$

In [1]:
import numpy as np
from numpy import matmul
import sklearn
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.linear_model
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt

###  Importing the dataset

In [2]:
# Import the boston dataset from sklearn
# Load dataset to some variable 
boston_data = load_boston()

In [3]:
#  Create X and Y variables - X holding the .data and Y holding .target 
X = boston_data.data
y = boston_data.target


#  Reshape Y to be a rank 2 matrix using y.reshape()
# y = y.reshape(int(y.size/2),2)
# print("y has been reshaped to " + str(y.shape))

# Observe the number of features and the number of labels
print('The number of features is: ', X.shape[1])
# Printing out the features
print('The features: ', boston_data.feature_names)
# The number of examples
print('The number of exampels in our dataset: ', X.shape[0])
# Observing the first 2 rows of the data
print(X[0:2])


The number of features is:  13
The features:  ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
The number of exampels in our dataset:  506
[[6.3200e-03 1.8000e+01 2.3100e+00 0.0000e+00 5.3800e-01 6.5750e+00
  6.5200e+01 4.0900e+00 1.0000e+00 2.9600e+02 1.5300e+01 3.9690e+02
  4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 6.4210e+00
  7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9690e+02
  9.1400e+00]]


We will also create polynomial feeatures for the dataset to test linear and ridge regression on data with d = 1 and data with d = 2. Feel free to increase the # of degress and see what effect it has on the training and test error. 

In [4]:
# Create a PolynomialFeatures object with degree = 2. Using PolynomialFeatures(degree=2)
# Transform X and save it into X_2 using poly.fit_transform(X)
# Simply copy Y into Y_2 
poly = PolynomialFeatures(2)
X_2 = poly.fit_transform(X)
y_2 = y

In [5]:
# the shape of X_2 and Y_2 - should be (506, 105) and (506, 1) respectively
print(X_2.shape)
print(y_2.shape)

(506, 105)
(506,)


In [6]:
# Define the get_coeff_ridge_normaleq function. Use the normal equation method.
# Return w values

def get_coeff_ridge_normaleq(X_train, y_train, alpha):
    # use np.linalg.pinv(...)
    X_T = np.transpose(X_train)
    w = matmul(matmul(np.linalg.pinv(matmul(X_T,X_train)+(alpha*np.identity(np.size(X_train,1)))),X_T),y_train)
    return w

In [7]:
# Define the get_coeff_ridge_normaleq function. Use the normal equation method.
# Return w values

def get_coeff_linear_normaleq(X_train, y_train):
    # use np.linalg.pinv(...)
    X_T = np.transpose(X_train)
    w = matmul(matmul(np.linalg.pinv(matmul(X_T,X_train)),X_T),y_train)
    return w

In [8]:
# Define the evaluate_err_ridge function.
# Return the train_error and test_error values


def evaluate_err(X_train, X_test, y_train, y_test, w): 
#     pred_train=prediction using w and X_train+np.mean(y_train)
    pred_train = matmul(w,np.transpose(X_train))
#     pred_test=prediction using w and X_test
    pred_test = matmul(w,np.transpose(X_test))
#     remember to add the mean back

    train_error= np.sum(np.square(pred_train - y_train))/int(y.size)
    test_error= np.sum(np.square(pred_test - y_test))/int(y.size)
    
    return train_error, test_error

In [9]:
# Finish writting the k_fold_cross_validation function. 
# Returns the average training error and average test error from the k-fold cross validation
# Sklearns K-Folds cross-validator: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

def k_fold_cross_validation(k, X, y, alpha=None):
    kf = KFold(n_splits=k, random_state=21, shuffle=True)
    total_E_val_test = 0
    total_E_val_train = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Centering the data so we do not need the intercept term (we could have also chose w_0=average y value)
        # Subtract y_train_mean from y_train and y_test
        y_train_mean = np.average(y_train)
        y_train = y_train - y_train_mean
        y_test = y_test - y_train_mean
        
        # Scaling the data matrix
        # Using scaler=preprocessing.StandardScaler().fit(...)
        # And scaler.transform(...)
        scaler=preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        # Determine the training error and the test error
        # Use get_coeff_linear_normaleq or get_coeff_ridge_normaleq to get w
        # And use evaluate_err()
        if alpha == None:
            w = get_coeff_linear_normaleq(X_train, y_train)
        else:
            w = get_coeff_ridge_normaleq(X_train, y_train, alpha)
        train_error, test_error = evaluate_err(X_train, X_test, y_train, y_test, w)
        total_E_val_test = total_E_val_test + test_error
        total_E_val_train = total_E_val_train +train_error
        


       ##############
    return  total_E_val_test, total_E_val_train
    


In [10]:
def print_error(alpha, total_E_val_test, total_E_val_train):
    print("----------{:.6}----------".format(alpha))
    print("Test Error: {} Train Error: {}".format(total_E_val_test, total_E_val_train))

In [11]:
# print the error for the both linear regression and ridge regression
# the error should include both training error and testing error

k = 23

# Error for linear regression
total_E_val_test, total_E_val_train = k_fold_cross_validation(k, X, y)
print_error("Linear", total_E_val_test, total_E_val_train/k)

# Error for Ridge Regression
for alpha in np.logspace(1,7, num=13):
    total_E_val_test, total_E_val_train = k_fold_cross_validation(k, X, y, alpha)
    print_error(alpha, total_E_val_test, total_E_val_train/k)




----------Linear----------
Test Error: 23.814099478324252 Train Error: 20.903355270040077
----------10.0----------
Test Error: 23.831212525393095 Train Error: 20.978438681623143
----------31.6228----------
Test Error: 24.0967636639064 Train Error: 21.326253537929905
----------100.0----------
Test Error: 25.231522741511704 Train Error: 22.619174073116447
----------316.228----------
Test Error: 29.093670513595224 Train Error: 26.634588774931718
----------1000.0----------
Test Error: 38.77449026538695 Train Error: 36.21614096894125
----------3162.28----------
Test Error: 53.80828954499053 Train Error: 50.845324814074424
----------10000.0----------
Test Error: 69.10554389305757 Train Error: 65.61090492154696
----------31622.8----------
Test Error: 78.67160621565678 Train Error: 74.81275173599427
----------1e+05----------
Test Error: 82.73042532923382 Train Error: 78.71260919918016
----------3.16228e+05----------
Test Error: 84.15615474914756 Train Error: 80.08201723413298
----------1e+06--

In [12]:
# Model with Polynomial Features

k=23

# Error for linear regression
total_E_val_test, total_E_val_train = k_fold_cross_validation(k, X_2, y_2)
print_error("Linear", total_E_val_test, total_E_val_train/k)

# Error for Ridge Regression
for alpha in np.logspace(1,7, num=13):
    total_E_val_test, total_E_val_train = k_fold_cross_validation(k, X_2, y_2, alpha)
    print_error(alpha, total_E_val_test, total_E_val_train/k)

----------Linear----------
Test Error: 12.32424061557486 Train Error: 5.65499187495009
----------10.0----------
Test Error: 13.387672506197465 Train Error: 9.597239786830333
----------31.6228----------
Test Error: 15.764457061218113 Train Error: 12.11395337179339
----------100.0----------
Test Error: 18.957402274368985 Train Error: 15.419017855937915
----------316.228----------
Test Error: 22.01267239266144 Train Error: 18.730018758978243
----------1000.0----------
Test Error: 25.942627911879896 Train Error: 22.98981030792135
----------3162.28----------
Test Error: 33.94987970614068 Train Error: 31.118480179006053
----------10000.0----------
Test Error: 47.012890618203926 Train Error: 44.012304097886734
----------31622.8----------
Test Error: 61.93124955396309 Train Error: 58.5949765844866
----------1e+05----------
Test Error: 74.31660686918063 Train Error: 70.5949619283571
----------3.16228e+05----------
Test Error: 80.9455591312574 Train Error: 76.98903717112036
----------1e+06------

In [13]:
# Getting parameters of the best model [Linear Regression with Polynomial Features of second order]

w = get_coeff_linear_normaleq(X_2, y_2)
print(w)

[ 2.89176201e-01 -6.96195792e+00  2.64067984e-01 -3.10232186e+00
 -2.61115550e-01 -7.98648586e-02  1.19457489e+01  7.42708817e-01
 -1.52258329e+01  3.31403243e+00 -1.25220413e-01  1.32411984e-01
  3.17545781e-02  8.91696273e-02  1.45253857e-03  1.67053032e-01
  2.22052033e-01  2.81094561e+00 -3.63747353e-01  1.56108868e-01
 -2.80410199e-03 -1.06722114e-01 -5.95086552e-02 -3.40385362e-03
  2.85411313e-01 -3.49399266e-04  2.12283479e-02 -2.88370318e-04
 -4.72323816e-03 -4.66579035e-02 -1.14311965e+00  5.70934279e-03
  4.37232955e-05 -1.18174261e-02 -3.74857158e-03  5.40368258e-04
 -7.16740379e-03  7.88659907e-04 -4.51780529e-03  2.78758895e-02
 -1.87410822e-01  1.04644822e+00  1.49957416e-01  4.30453942e-03
  8.77936473e-02 -6.42096125e-02  8.54014583e-04 -2.61454187e-02
  2.89148389e-03 -1.51426896e-02 -2.61095864e-01 -1.60571716e+00
 -4.36017523e+00  8.16306787e-02  1.81008721e+00 -5.76672238e-01
 -1.26071180e-03  7.25845292e-01  3.98585840e-02 -4.56952828e-01
 -7.68771253e-01  7.06943